In [1]:
import os 
from rich import print 
import torch 
from torch.utils.data import Dataset, DataLoader 

data_dir = '/home/yunzhi/yunzhi/datasets'
train_path = '/home/yunzhi/yunzhi/datasets/Train'
test_path = '/home/yunzhi/yunzhi/datasets'

# Resizing the images to 30x30x3
IMG_HEIGHT = 224
IMG_WIDTH = 224
channels = 3
NUM_CATEGORIES = len(os.listdir(train_path))
print(NUM_CATEGORIES)

# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }
print(classes)

43

{
    0: 'Speed limit (20km/h)',
    1: 'Speed limit (30km/h)',
    2: 'Speed limit (50km/h)',
    3: 'Speed limit (60km/h)',
    4: 'Speed limit (70km/h)',
    5: 'Speed limit (80km/h)',
    6: 'End of speed limit (80km/h)',
    7: 'Speed limit (100km/h)',
    8: 'Speed limit (120km/h)',
    9: 'No passing',
    10: 'No passing veh over 3.5 tons',
    11: 'Right-of-way at intersection',
    12: 'Priority road',
    13: 'Yield',
    14: 'Stop',
    15: 'No vehicles',
    16: 'Veh > 3.5 tons prohibited',
    17: 'No entry',
    18: 'General caution',
    19: 'Dangerous curve left',
    20: 'Dangerous curve right',
    21: 'Double curve',
    22: 'Bumpy road',
    23: 'Slippery road',
    24: 'Road narrows on the right',
    25: 'Road work',
    26: 'Traffic signals',
    27: 'Pedestrians',
    28: 'Children crossing',
    29: 'Bicycles crossing',
    30: 'Beware of ice/snow',
    31: 'Wild animals crossing',
    32: 'End speed + passing limits',
    33: 'Turn right ahead',
    34: 'Turn left ahead',
    35: 'Ahead only',
    36: 'Go straight or right',
    37: 'Go straight or left',
    38: 'Keep right',
    39: 'Keep left',
    40: 'Roundabout mandatory',
    41: 'End of no passing',
    42: 'End no passing veh > 3.5 tons'
}

In [11]:
from PIL import Image 
from glob import glob
import numpy as np 

class GTSRB(Dataset):
    def __init__(self,
                data_dir='/home/yunzhi/yunzhi/datasets',
                train_path='/home/yunzhi/yunzhi/datasets/Train',
                test_path='/home/yunzhi/yunzhi/datasets',
                mode='train') -> None:
        super().__init__()
        self.data_dir = data_dir,
        self.train_path = train_path
        self.test_path = test_path
        self.data_infos = []
        print("load annos:")
        self.load_annotations()

    def show(self, index):
        data_info = self.data_infos[index].copy()
        path = data_info['path']
        label = data_info['label']
        print(label)
        return Image.open(path)

    def load_annotations(self):
        # load Annotation
        self.data_infos = []
        for cat in classes:
            label = classes[cat]
            img_dir = os.path.join(self.train_path, str(cat))
            img_names = glob(os.path.join(img_dir, "*.png"))
            # print(img_dir, img_names)
            for name in img_names:
                data_info = dict(
                    path=os.path.join(img_dir, name),
                    label=label
                )
                self.data_infos.append(data_info)
        return 

    def __getitem__(self, index):
        data_info = self.data_infos[index].copy()
        path = data_info['path']
        label = data_info['label']
        return dict(
            img=Image.open(path),
            label=label,
            path=path
        )
    
    def __len__(self):
        return len(self.data_infos)
dataset = GTSRB()

load annos:

In [12]:
dataset[0]

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=76x75>,
 'label': 'Speed limit (20km/h)',
 'path': '/home/yunzhi/yunzhi/datasets/Train/0/00000_00003_00026.png'}

In [13]:
from Flamingo.models.batchprocessor import CLIPBatchProcessor
from Flamingo.lora_tuning import get_tokenizer
from Flamingo.models.modeling_clip import get_clip_vision_encoder_and_processor
from tqdm import tqdm 

In [4]:
vision_encoder, image_processor = get_clip_vision_encoder_and_processor()
vision_encoder = vision_encoder.cuda()

In [5]:
batch_processor = CLIPBatchProcessor(vision_encoder=vision_encoder,
                                      image_processor=image_processor)


In [17]:
def collate_fn(batch):
    imgs = []
    # labels = []
    paths = []
    for data in batch:
        imgs.append(data['img'])
        paths.append(data['path'])
        # labels.append(data['label'])
    return dict(
        path=paths,
        img=imgs
    )

In [18]:
dataloader = DataLoader(dataset=dataset, batch_size=24, shuffle=False, num_workers=12, collate_fn=collate_fn)

In [19]:
for data in tqdm(dataloader):
    imgs = data['img']
    paths = data['path']
    out = batch_processor(imgs)    # [B, 1, 1, V, D]
    
    # save to disk
    for i in range(len(paths)):
        path = paths[i] + ".pth"
        torch.save(out[i][None, ...].cpu(),path)

  0%|          | 0/1634 [00:00<?, ?it/s]

tensor([[[[-5.9519e-01, -5.9519e-01, -5.9519e-01,  ..., -8.4336e-01,
           -8.2877e-01, -8.2877e-01],
          [-5.6599e-01, -5.6599e-01, -5.6599e-01,  ..., -8.4336e-01,
           -8.2877e-01, -8.2877e-01],
          [-4.9300e-01, -4.9300e-01, -4.9300e-01,  ..., -8.4336e-01,
           -8.2877e-01, -8.1417e-01],
          ...,
          [ 4.9969e-01,  5.1429e-01,  5.2889e-01,  ..., -1.8644e-01,
           -1.8644e-01, -2.0103e-01],
          [ 4.9969e-01,  5.1429e-01,  5.2889e-01,  ..., -1.5724e-01,
           -1.7184e-01, -1.8644e-01],
          [ 4.9969e-01,  5.1429e-01,  5.2889e-01,  ..., -1.4264e-01,
           -1.7184e-01, -1.8644e-01]],

         [[-5.8149e-01, -6.1151e-01, -5.9650e-01,  ..., -7.6158e-01,
           -7.6158e-01, -7.4658e-01],
          [-5.5148e-01, -5.6648e-01, -5.6648e-01,  ..., -7.6158e-01,
           -7.6158e-01, -7.4658e-01],
          [-4.6143e-01, -4.7644e-01, -4.9144e-01,  ..., -7.6158e-01,
           -7.6158e-01, -7.4658e-01],
          ...,
          [ 6.7916e-01,  6.7916e-01,  6.9417e-01,  ...,  1.8820e-02,
            4.8835e-02,  6.3843e-02],
          [ 6.7916e-01,  6.7916e-01,  6.9417e-01,  ...,  4.8835e-02,
            6.3843e-02,  6.3843e-02],
          [ 6.7916e-01,  6.7916e-01,  6.9417e-01,  ...,  6.3843e-02,
            6.3843e-02,  6.3843e-02]],

         [[-3.4261e-01, -3.5683e-01, -3.5683e-01,  ..., -5.4170e-01,
           -5.2748e-01, -5.2748e-01],
          [-3.1417e-01, -3.2839e-01, -3.2839e-01,  ..., -5.4170e-01,
           -5.2748e-01, -5.2748e-01],
          [-2.4307e-01, -2.5729e-01, -2.5729e-01,  ..., -5.4170e-01,
           -5.2748e-01, -5.2748e-01],
          ...,
          [ 9.0875e-01,  9.0875e-01,  9.0875e-01,  ...,  3.6839e-01,
            3.9683e-01,  4.1105e-01],
          [ 8.9453e-01,  9.0875e-01,  9.0875e-01,  ...,  3.9683e-01,
            4.1105e-01,  4.1105e-01],
          [ 8.9453e-01,  9.0875e-01,  9.0875e-01,  ...,  4.1105e-01,
            4.1105e-01,  4.1105e-01]]],


        [[[ 4.5590e-01,  4.5590e-01,  4.5590e-01,  ...,  3.8290e-01,
            3.6830e-01,  3.6830e-01],
          [ 4.4130e-01,  4.4130e-01,  4.4130e-01,  ...,  3.3911e-01,
            3.2451e-01,  3.2451e-01],
          [ 4.2670e-01,  4.2670e-01,  4.2670e-01,  ...,  2.9531e-01,
            2.8071e-01,  2.8071e-01],
          ...,
          [ 1.9303e+00,  1.9303e+00,  1.9303e+00,  ...,  1.9303e+00,
            1.9303e+00,  1.9303e+00],
          [ 1.9303e+00,  1.9303e+00,  1.9303e+00,  ...,  1.9303e+00,
            1.9303e+00,  1.9303e+00],
          [ 1.9303e+00,  1.9303e+00,  1.9303e+00,  ...,  1.9303e+00,
            1.9303e+00,  1.9303e+00]],

         [[-1.1368e+00, -1.1368e+00, -1.1368e+00,  ..., -2.9634e-01,
           -2.9634e-01, -2.9634e-01],
          [-1.1368e+00, -1.1368e+00, -1.1368e+00,  ..., -2.5132e-01,
           -2.5132e-01, -2.5132e-01],
          [-1.1518e+00, -1.1518e+00, -1.1518e+00,  ..., -2.0630e-01,
           -2.0630e-01, -2.0630e-01],
          ...,
          [ 2.0749e+00,  2.0749e+00,  2.0749e+00,  ...,  2.0749e+00,
            2.0749e+00,  2.0749e+00],
          [ 2.0749e+00,  2.0749e+00,  2.0749e+00,  ...,  2.0749e+00,
            2.0749e+00,  2.0749e+00],
          [ 2.0749e+00,  2.0749e+00,  2.0749e+00,  ...,  2.0749e+00,
            2.0749e+00,  2.0749e+00]],

         [[-9.5408e-01, -9.6830e-01, -9.6830e-01,  ..., -2.7151e-01,
           -2.7151e-01, -2.7151e-01],
          [-9.8252e-01, -9.9674e-01, -9.9674e-01,  ..., -2.1463e-01,
           -2.1463e-01, -2.1463e-01],
          [-1.0110e+00, -1.0252e+00, -1.0252e+00,  ..., -1.4353e-01,
           -1.4353e-01, -1.4353e-01],
          ...,
          [ 2.1459e+00,  2.1459e+00,  2.1459e+00,  ...,  2.1032e+00,
            2.1175e+00,  2.1175e+00],
          [ 2.1459e+00,  2.1459e+00,  2.1459e+00,  ...,  2.1317e+00,
            2.1317e+00,  2.1317e+00],
          [ 2.1459e+00,  2.1459e+00,  2.1459e+00,  ...,  2.1459e+00,
            2.1459e+00,  2.1459e+00]]],


        [[[ 9.6684e-01,  9.9604e-01,  9.81

torch.Size([24, 3, 224, 224])

  0%|          | 0/1634 [00:00<?, ?it/s]


In [13]:
# isinstance(data[0], Image.Image)

True